# Original DRIS

Diagnosis and Recommendation Integrated System (DRIS) implemented according to "Diagnosis and Recommendation Integrated System (DRIS)" by J.L Walworth and M.E. Sumner. 

Questions:

- Calculate DRIS to compare two populations or to compare each sample plant with the target population?
- Is it the mean of the ratios or the ratios of the mean, the difference would be the factor number of plants

In [193]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations

In [194]:
data_diagnosed_dict = {
    'P': [12,11,10, 9],
    'Mg': [2,2,3,3],
    'N': [1,2,1,2],
    'Ca': [3,4,3,3],
    'Mn': [4,5,4,2]
}

data_optimum_dict = {
    'p': [12,11,10],
    'mg': [2,1,3],
    'n': [2,1,1],
    'ca': [4,4,3],
    'mn': [5,4,4],
}

df_diagnosed = pd.DataFrame(data_diagnosed_dict)
df_optimum = pd.DataFrame(data_optimum_dict)

## Calculate $f(A/B)$ 

\begin{align}
    f(A/B) = 
    \begin{cases}
    \biggl(\frac{A/B}{a/b} - 1\biggr)\frac{1000}{CV} & A/B \ge a/b\\
    \biggl(1 - \frac{a/b}{A/B}\biggr)\frac{1000}{CV} & A/B < a/b
    \end{cases}
\end{align}

- $A/B$ are the ratios of the means two elements (A and B) in the tissue of the pant being diagnosed.
- $a/b$ are the ratios of the means for the optimal (high yield) population. 
- CV is the coefficient of variation of the optimal values $a$ and $b$: $CV = \sigma/\mu$ 

**Comments:**
- Is a/b (resp. A/B) the mean of the ratios or the ration of the means? This is not clear to me!!!
- In the poplication of Jones the value is 100 not 1000, but in subsequent publications it is 1000.
- Is the CV (coefficient of variation) in percent or not (in the publication of jones it certainly is)

Calculate ratios A/B and a/b 

In [ ]:
def calculate_ratios(df):
    '''Calculate the rations of every combination'''
    ratios = {}
    for col1, col2 in combinations(df.columns, 2):
        ratio_name1 = f"{col1}/{col2}"
        ratios[ratio_name1] = df[col1] / df[col2]
    return pd.DataFrame(ratios)

df_optimum_ratios = calculate_ratios(df_optimum)
df_diagnosed_ratios = calculate_ratios(df_diagnosed)

df_diagnosed_ratios.head()

,P/Mg,P/N,P/Ca,P/Mn,Mg/N,Mg/Ca,Mg/Mn,N/Ca,N/Mn,Ca/Mn
0,1.791759,2.484907,1.386294,1.098612,0.693147,-0.405465,-0.693147,-1.098612,-1.386294,-0.287682
1,1.704748,1.704748,1.011601,0.788457,0.000000,-0.693147,-0.916291,-0.693147,-0.916291,-0.223144
2,1.203973,2.302585,1.203973,0.916291,1.098612,0.000000,-0.287682,-1.098612,-1.386294,-0.287682
3,1.098612,1.504077,1.098612,1.504077,0.405465,0.000000,0.405465,-0.405465,0.000000,0.405465


### Calculate coefficient of variation of the optimum values

In [196]:
def calculate_CV(df_ratios):
    cv_dict = {}
    for ratio_name, ratio_values in df_ratios.items():
        mean_ratio = ratio_values.mean()
        std_ratio = ratio_values.std()
        cv = (std_ratio / mean_ratio) * 100  # percentage or not? properbly percentage
        cv_dict[ratio_name] = [cv]
    return pd.DataFrame(cv_dict)

df_optimum_CV = calculate_CV(df_optimum_ratios)

### Calculate rations of the mean

In [197]:
# check the order!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
df_op = pd.DataFrame(df_optimum.mean()).T
df_optimum_mean_ratios = calculate_ratios(df_op)

df_di = pd.DataFrame(df_diagnosed.mean()).T
df_diagnosed_mean_ratios = calculate_ratios(df_di)

df_diagnosed_mean_ratios.head()

,P/Mg,P/N,P/Ca,P/Mn,Mg/N,Mg/Ca,Mg/Mn,N/Ca,N/Mn,Ca/Mn
0,1.435085,1.94591,1.17272,1.029619,0.510826,-0.262364,-0.405465,-0.77319,-0.916291,-0.143101


In [198]:
df_optimum_mean_ratios.head()

,p/mg,p/n,p/ca,p/mn,mg/n,mg/ca,mg/mn,n/ca,n/mn,ca/mn
0,1.704748,2.110213,1.098612,0.931558,0.405465,-0.606136,-0.77319,-1.011601,-1.178655,-0.167054


### Calculate nutrient function

In [199]:
def f(df_diagnosed_mean_ratios, df_optimum_mean_ratios, df_optimum_CV):
    names = df_diagnosed_mean_ratios.columns
    diagnosed_ratios = df_diagnosed_mean_ratios.to_numpy()
    optimum_mean_ratios = df_optimum_mean_ratios.to_numpy()
    optimum_CV = df_optimum_CV.to_numpy()

    def f_single(diagnosed_ratio, optimum_mean_ratio, CV):
        if diagnosed_ratio == 0:
            return float('inf')  # Return inf or some large number to handle zero division
        if diagnosed_ratio >= optimum_mean_ratio:
            f = ((diagnosed_ratio / optimum_mean_ratio) - 1) * 1000 / CV
        else:
            f = (1 - (optimum_mean_ratio / diagnosed_ratio)) * 1000 / CV
        return [f]

    f_dict = dict()
    for i in range(len(names)):
        f_dict[f"f({names[i]})"] = f_single(diagnosed_ratios[0, i], optimum_mean_ratios[0, i], optimum_CV[0, i])
    return pd.DataFrame(f_dict)


# Example usage
# Assuming df_diagnosed_mean_ratios, df_optimum_mean_ratios, df_optimum_CV are defined DataFrames with similar structure
df_f = f(df_diagnosed_mean_ratios, df_optimum_mean_ratios, df_optimum_CV)
df_f.head()


,f(P/Mg),f(P/N),f(P/Ca),f(P/Mn),f(Mg/N),f(Mg/Ca),f(Mg/Mn),f(N/Ca),f(N/Mn),f(Ca/Mn)
0,-5.65902,-5.606043,7.73583,14.080464,1.50025,5.671527,7.449758,7.169381,10.086735,1.617407


## Calculate DRIS index


$\mathrm{A_{index}} = \frac{1}{z} \bigl[f(A/B) + f(A/C) + f(A/D) + \cdots + f(A/N)\bigr]$

In [200]:
def create_index_string(index_element, df_diagnosed):
    ''' Create string representing the the equation to calculate the DRIS index '''
    elements = df_diagnosed.columns
    result_string = f'I_{elements[index_element]} = 1/{(len(elements)-1)} ('
    for i, element in enumerate(elements):
        if index_element < i:
            result_string += (f' + f({elements[index_element]}/{element})')
        elif index_element > i:
            result_string += (f' - f({element}/{elements[index_element]})')
    return result_string + ')'

result_string = create_index_string(2,df_diagnosed)
result_string

'I_N = 1/4 ( - f(P/N) - f(Mg/N) + f(N/Ca) + f(N/Mn))'

In [201]:
def calculate_index_value(index_element, df_diagnosed, df_f):
    ''' Calculates the DRIS index '''
    f_dict = df_f.to_dict('index')[0]
    elements = df_diagnosed.columns
    result = 0
    for i, element in enumerate(elements):
        if index_element < i:
            result += f_dict[f'f({elements[index_element]}/{element})']
        elif index_element > i:
            result -= f_dict[f'f({element}/{elements[index_element]})']
    return result/(len(elements)-1)

In [202]:
import pandas as pd

def calculate_all_index_values(df_diagnosed, df_f):
    """
    Calculate the index values for each element in the input DataFrame.
    
    Parameters:
        df_diagnosed (pd.DataFrame): DataFrame where each column represents a diagnosed nutrient.
        df_f (pd.DataFrame): DataFrame with calculated f(A/B) or f(B/A) values for each element ratio.
        
    Returns:
        pd.DataFrame: DataFrame with index values for each element, where each row corresponds to an element.
    """
    elements = df_diagnosed.columns
    results_dict = {}
    
    # Calculate index value for each element and store in results_dict
    for i, element in enumerate(elements):
        # Assuming calculate_index_value is a function that calculates the index for a single element
        index_value = calculate_index_value(i, df_diagnosed, df_f)
        results_dict[element] = index_value
    
    # Convert results_dict to a DataFrame with the specified format
    results_df = pd.DataFrame(list(results_dict.items()), columns=["Element", "I_DRIS"])
    results_df.set_index("Element", inplace=True)
    
    return results_df

# Example usage
# df_diagnosed and df_f should be DataFrames with appropriate data for the calculation
DRIS_indices = calculate_all_index_values(df_diagnosed, df_f)
DRIS_indices.head()


,I_DRIS
Element,
P,2.637808
Mg,5.070139
N,5.340477
Ca,-4.739833
Mn,-8.308591


## Putting it together

In [203]:
def calculate_DRIS_index(df_diagnosed, df_optimum):
    # calculate ratios
    df_optimum_ratios = calculate_ratios(df_optimum)
    # df_diagnosed_ratios = calculate_ratios(df_diagnosed)
    # calculate CV
    df_optimum_CV = calculate_CV(df_optimum_ratios)
    # calculate mean _ratios
    df_op = pd.DataFrame(df_optimum.mean()).T
    df_optimum_mean_ratios = calculate_ratios(df_op)
    df_di = pd.DataFrame(df_diagnosed.mean()).T
    df_diagnosed_mean_ratios = calculate_ratios(df_di)
    # calculate f
    df_f = f(df_diagnosed_mean_ratios, df_optimum_mean_ratios, df_optimum_CV)
    # calculate indices
    DRIS_indices = calculate_all_index_values(df_diagnosed, df_f)
    return DRIS_indices

DRIS = calculate_DRIS_index(df_diagnosed, df_optimum)
DRIS.head()


,I_DRIS
Element,
P,2.637808
Mg,5.070139
N,5.340477
Ca,-4.739833
Mn,-8.308591


In [204]:
def calculate_DRIS_index_mean_of_ratios(df_diagnosed, df_optimum):
    # calculate ratios
    df_optimum_ratios = calculate_ratios(df_optimum)
    # df_diagnosed_ratios = calculate_ratios(df_diagnosed)
    # calculate CV
    df_optimum_CV = calculate_CV(df_optimum_ratios)
    # calculate mean _ratios
    df_optimum_ratios = calculate_ratios(df_optimum)
    df_optimum_mean_ratios = pd.DataFrame(df_optimum_ratios.mean()).T
    
    df_diagnosed_ratios = calculate_ratios(df_diagnosed)
    df_diagnosed_mean_ratios = pd.DataFrame(df_diagnosed_ratios.mean()).T
    # calculate f
    df_f = f(df_diagnosed_mean_ratios, df_optimum_mean_ratios, df_optimum_CV)
    # calculate indices
    DRIS_indices = calculate_all_index_values(df_diagnosed, df_f)
    return DRIS_indices

DRIS = calculate_DRIS_index_mean_of_ratios(df_diagnosed, df_optimum)
DRIS.head()

,I_DRIS
Element,
P,3.745126
Mg,6.263545
N,5.170348
Ca,-3.833406
Mn,-11.345613


## Nurtional Balance Index average (NBIa)

In [205]:
def calculate_NBIa(df_dris):
    """ This value is for the whole population """
    n = len(df_dris.values[0])
    return np.sum(np.abs(df_dris.values[0]))/n

def calculate_NBI(df_dris):
    """ This value is for the whole population """
    return np.sum(np.abs(df_dris.values))

calculate_NBIa(DRIS)

np.float64(3.7451260818181815)

## Interpretation

In [206]:
def interpret(df_dris):
    NBIa = calculate_NBIa(df_dris)
    print(f"NBIa ={np.round(NBIa,2)}")
    for i, dris_index in enumerate(df_dris.values[0]):
        if dris_index < NBIa and dris_index < 0 and dris_index and dris_index == min(df_dris.values[0]):
            print(f"{df_dris.columns[i]} Deficiency")
        elif dris_index < NBIa and dris_index < 0:
            print(f"{df_dris.columns[i]} Deficiency-prone")
        elif np.round(dris_index,0) == np.round(NBIa,0):
            print(f"{df_dris.columns[i]} Sufficient")
        elif dris_index > NBIa and dris_index > 0 and dris_index == max(df_dris.values[0]):
            print(f"{df_dris.columns[i]} Excess")
        elif dris_index > NBIa and dris_index > 0:
            print(f"{df_dris.columns[i]} Excess-prone")
        elif dris_index > 0 and dris_index < NBIa:
            print(f"{df_dris.columns[i]} dris_index > 0 but dris_index < NBIa")
        else:
            print(f"{df_dris.columns[i]} dris_index < 0 but dris_index > NBIa")
            

interpret(DRIS)



NBIa =3.75
I_DRIS Sufficient
